# Neural DSL on Kaggle - Comprehensive Tutorial

This notebook demonstrates how to use Neural DSL in Kaggle environments with advanced features, error handling, and cloud optimizations.

## Table of Contents
1. [Installation](#installation)
2. [Environment Setup](#environment-setup)
3. [Basic Model Compilation](#basic-model-compilation)
4. [Model Training](#model-training)
5. [Model Visualization](#model-visualization)
6. [Advanced Features](#advanced-features)
7. [Error Handling](#error-handling)
8. [Cloud Optimizations](#cloud-optimizations)
9. [Interactive Debugging](#interactive-debugging)
10. [Cleanup](#cleanup)

## 1. Installation <a id='installation'></a>

First, we need to install Neural DSL. This may take a few minutes.

In [ ]:
# Install Neural DSL
!pip install -q git+https://github.com/Lemniscate-SHA-256/Neural.git

# Install optional dependencies for cloud features
!pip install -q pyngrok

print("✓ Installation complete!")

## 2. Environment Setup <a id='environment-setup'></a>

Let's initialize the cloud executor and check our environment.

In [ ]:
import json

from neural.cloud.cloud_execution import CloudExecutor


# Initialize with custom settings
executor = CloudExecutor(
    timeout=600,  # 10 minutes timeout
    retry_attempts=3  # Retry failed operations 3 times
)

# Get comprehensive environment information
env_info = executor.get_environment_info()
print("Environment Information:")
print(json.dumps(env_info, indent=2))

print(f"\n✓ Detected environment: {executor.environment}")
print(f"✓ GPU available: {executor.is_gpu_available}")
print(f"✓ Optimization level: {executor.optimization_level}")

## 3. Basic Model Compilation <a id='basic-model-compilation'></a>

Let's define and compile a simple CNN model for MNIST classification.

In [ ]:
# Define a Neural DSL model
dsl_code = """
network MnistCNN {
    input: (28, 28, 1)
    layers:
        Conv2D(32, (3, 3), "relu")
        MaxPooling2D((2, 2))
        Conv2D(64, (3, 3), "relu")
        MaxPooling2D((2, 2))
        Flatten()
        Dense(128, "relu")
        Dropout(0.5)
        Dense(10, "softmax")
    loss: "categorical_crossentropy"
    optimizer: Adam(learning_rate=0.001)
}
"""

print("Model Definition:")
print(dsl_code)

# Compile to TensorFlow with validation
try:
    tf_model_path = executor.compile_model(
        dsl_code, 
        backend='tensorflow',
        validate=True
    )
    print(f"\n✓ Model compiled successfully to: {tf_model_path}")
    
    # Display first 50 lines of generated code
    with open(tf_model_path, 'r') as f:
        lines = f.readlines()[:50]
        print("\nGenerated Code (first 50 lines):")
        print(''.join(lines))
        if len(f.readlines()) > 50:
            print("\n... (truncated)")
            
except Exception as e:
    print(f"✗ Compilation failed: {e}")

## 4. Model Training <a id='model-training'></a>

Now let's run the compiled model with custom training parameters.

In [ ]:
# Run the model with error handling
results = executor.run_model(
    tf_model_path,
    dataset='MNIST',
    epochs=3,
    batch_size=64,
    timeout=300  # 5 minutes timeout
)

# Check results
if results['success']:
    print("✓ Model execution successful!")
    print("\nOutput:")
    print(results['stdout'])
else:
    print(f"✗ Model execution failed: {results['error']}")
    print(f"Error type: {results.get('error_type', 'unknown')}")
    if results.get('stderr'):
        print(f"\nError details:\n{results['stderr']}")

## 5. Model Visualization <a id='model-visualization'></a>

Visualize the model architecture.

In [ ]:
try:
    # Generate visualization
    viz_path = executor.visualize_model(dsl_code, output_format='png')
    print(f"✓ Visualization saved to: {viz_path}")
    
    # Display the visualization
    from IPython.display import Image, display
    display(Image(filename=viz_path))
    
except Exception as e:
    print(f"✗ Visualization failed: {e}")

## 6. Advanced Features <a id='advanced-features'></a>

### 6.1 Multi-Backend Compilation

Compile the same model to different backends.

In [ ]:
backends = ['tensorflow', 'pytorch']
compiled_models = {}

for backend in backends:
    try:
        model_path = executor.compile_model(dsl_code, backend=backend)
        compiled_models[backend] = model_path
        print(f"✓ {backend.capitalize()} compilation successful: {model_path}")
    except Exception as e:
        print(f"✗ {backend.capitalize()} compilation failed: {e}")

print(f"\nSuccessfully compiled to {len(compiled_models)} backend(s)")

### 6.2 Complex Model with Residual Connections

In [ ]:
# Define a more complex model
complex_dsl = """
network ResNetBlock {
    input: (32, 32, 3)
    layers:
        Conv2D(64, (3, 3), "relu", padding="same")
        BatchNormalization()
        Conv2D(64, (3, 3), "relu", padding="same")
        BatchNormalization()
        MaxPooling2D((2, 2))
        Dropout(0.25)
        
        Conv2D(128, (3, 3), "relu", padding="same")
        BatchNormalization()
        Conv2D(128, (3, 3), "relu", padding="same")
        BatchNormalization()
        MaxPooling2D((2, 2))
        Dropout(0.25)
        
        GlobalAveragePooling2D()
        Dense(256, "relu")
        Dropout(0.5)
        Dense(10, "softmax")
    loss: "categorical_crossentropy"
    optimizer: Adam(learning_rate=0.001)
}
"""

try:
    complex_model_path = executor.compile_model(complex_dsl, backend='tensorflow')
    print(f"✓ Complex model compiled: {complex_model_path}")
except Exception as e:
    print(f"✗ Complex model compilation failed: {e}")

## 7. Error Handling <a id='error-handling'></a>

Demonstrate robust error handling with intentional errors.

In [ ]:
# Test 1: Empty DSL code
print("Test 1: Empty DSL code")
try:
    executor.compile_model("", backend='tensorflow')
    print("✗ Should have failed")
except Exception as e:
    print(f"✓ Caught expected error: {type(e).__name__}")

# Test 2: Invalid model file path
print("\nTest 2: Non-existent model file")
try:
    executor.run_model('/invalid/path/model.py', dataset='MNIST')
    print("✗ Should have failed")
except Exception as e:
    print(f"✓ Caught expected error: {type(e).__name__}")

# Test 3: Invalid DSL syntax
print("\nTest 3: Invalid DSL syntax")
invalid_dsl = "network Invalid { invalid syntax }"
try:
    executor.compile_model(invalid_dsl, backend='tensorflow')
    print("✗ Should have failed")
except Exception as e:
    print(f"✓ Caught expected error: {type(e).__name__}")

print("\n✓ All error handling tests passed!")

## 8. Cloud Optimizations <a id='cloud-optimizations'></a>

Neural DSL automatically applies cloud-specific optimizations based on your environment.

In [ ]:
import os


# Display active optimizations
print("Active Cloud Optimizations:")
print(f"- Environment: {executor.environment}")
print(f"- Optimization Level: {executor.optimization_level}")
print(f"- GPU Memory Growth: {os.environ.get('TF_FORCE_GPU_ALLOW_GROWTH', 'not set')}")
print(f"- TensorFlow Log Level: {os.environ.get('TF_CPP_MIN_LOG_LEVEL', 'not set')}")
print(f"- PyTorch CUDA Allocation: {os.environ.get('PYTORCH_CUDA_ALLOC_CONF', 'not set')}")
print(f"- Unbuffered Python Output: {os.environ.get('PYTHONUNBUFFERED', 'not set')}")

if executor.is_gpu_available:
    print("\n✓ GPU optimizations enabled")
else:
    print("\n✓ CPU optimizations enabled")

## 9. Interactive Debugging <a id='interactive-debugging'></a>

Start the NeuralDbg dashboard with an ngrok tunnel for remote access.

In [ ]:
# Start the dashboard (optional - requires ngrok)
try:
    dashboard_info = executor.start_debug_dashboard(
        dsl_code,
        backend='tensorflow',
        setup_tunnel=True,
        port=8050
    )
    
    if dashboard_info['status'] == 'running':
        print("✓ Dashboard started successfully")
        print(f"Dashboard URL: {dashboard_info['dashboard_url']}")
        print(f"Tunnel URL: {dashboard_info['tunnel_url']}")
        print(f"Process ID: {dashboard_info['process_id']}")
        
        # Display a clickable link
        from IPython.display import HTML, display
        display(HTML(f"<a href='{dashboard_info['tunnel_url']}' target='_blank'>Open NeuralDbg Dashboard</a>"))
    else:
        print(f"✗ Dashboard failed to start: {dashboard_info.get('error', 'Unknown error')}")
        
except Exception as e:
    print(f"✗ Could not start dashboard: {e}")

### Start No-Code Interface

Launch the Neural No-Code interface for visual model building.

In [ ]:
# Start the No-Code interface (optional)
try:
    nocode_info = executor.start_nocode_interface(
        port=8051,
        setup_tunnel=True
    )
    
    if nocode_info['status'] == 'running':
        print("✓ No-Code interface started successfully")
        print(f"Interface URL: {nocode_info['interface_url']}")
        print(f"Tunnel URL: {nocode_info['tunnel_url']}")
        
        # Display a clickable link
        from IPython.display import HTML, display
        display(HTML(f"<a href='{nocode_info['tunnel_url']}' target='_blank'>Open Neural No-Code Interface</a>"))
    else:
        print(f"✗ No-Code interface failed to start: {nocode_info.get('error', 'Unknown error')}")
        
except Exception as e:
    print(f"✗ Could not start No-Code interface: {e}")

## 10. Cleanup <a id='cleanup'></a>

Clean up temporary files and processes when you're done.

In [ ]:
# Clean up all resources
executor.cleanup()
print("✓ Cleanup complete!")

## Summary

In this notebook, you learned how to:

1. ✓ Install and configure Neural DSL on Kaggle
2. ✓ Check environment information and GPU availability
3. ✓ Compile Neural DSL models to different backends
4. ✓ Train models with custom parameters and error handling
5. ✓ Visualize model architectures
6. ✓ Use advanced features like multi-backend compilation
7. ✓ Handle errors gracefully
8. ✓ Leverage cloud-specific optimizations
9. ✓ Access debugging dashboards remotely via ngrok
10. ✓ Clean up resources properly

For more information, visit the [Neural DSL documentation](https://github.com/Lemniscate-SHA-256/Neural).